In [1]:
import numpy 
numpy.random.seed(1337)

In [2]:
import h5py
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, MaxPooling2D, Dropout, AveragePooling2D, GlobalAveragePooling2D
from keras.layers import Conv2D, Embedding
from keras.optimizers import SGD, rmsprop
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU
from keras_contrib.layers.advanced_activations import PELU, SReLU

Using TensorFlow backend.


In [3]:
batch_size=10
num_classes=37
epochs=25

In [4]:
import os
from PIL import Image
import matplotlib.image as mpimg

In [5]:
os.chdir("E:");
#path="E:/Dataset_Final(March)/dum";
path="E:/Dummy/Level_1-12";
#path="E:/Dataset_Final(April)/six";
classes=os.listdir(path)
x=[]#Datapoints 
y=[]#labels 
for fol in classes:
    imgfiles=os.listdir(path+u'\\'+fol);
    for img in imgfiles:
        im=mpimg.imread(path+u'\\'+fol+u'\\'+img);
        x.append(im)
        y.append(fol)
x=numpy.array(x)
y=numpy.array(y)

In [6]:
print(x.shape)
print(y.shape)
x=x.reshape((-1,100,100,1))

(285, 100, 100)
(285,)


In [7]:
n=x.shape[0]
randomize=numpy.arange(n)
numpy.random.shuffle(randomize)
randomize
x=x[randomize]
y=y[randomize]

In [8]:
test_split=round(n*2/3)
x_train=x[:test_split]
y_train=y[:test_split]
x_test=x[test_split:]
y_test=y[test_split:]

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(190, 100, 100, 1)
(190,)
(95, 100, 100, 1)
(95,)


In [9]:
y_train=keras.utils.to_categorical(y_train, num_classes)
y_test=keras.utils.to_categorical(y_test, num_classes)
y_train[:1]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.]])

In [10]:
# from keras import applications
# model = applications.VGG16(weights='imagenet', include_top=False)
# print('Model loaded.')

In [46]:
# inputs=Input(shape=(100,100,1))

# x=Conv2D(16, (8, 8), padding='same')(inputs)
# x=SReLU()(x)
# x=MaxPooling2D(pool_size=(2,2))(x)


# x=Conv2D(32, (6, 6), padding='same')(x)
# x=SReLU()(x)
# x=MaxPooling2D(pool_size=(2,2))(x)


# x=Conv2D(64, (4, 4), padding='same')(x)
# x=SReLU()(x)
# x=MaxPooling2D(pool_size=(2,2))(x)

# x=Conv2D(128, (3, 3), padding='same')(x)
# x=SReLU()(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Dropout(0.2)(x)
# x=Flatten()(x)
# x=Dense(num_classes)(x)
# output=Activation('softmax')(x)
# model=Model([inputs], output)

In [56]:
inputs=Input(shape=(100,100,1))

x=Conv2D(16, (8, 8), padding='same')(inputs)
x=SReLU()(x)
x=MaxPooling2D(pool_size=(2,2))(x)


x=Conv2D(32, (7, 7), padding='same')(x)
x=SReLU()(x)
x=MaxPooling2D(pool_size=(2,2))(x)


x=Conv2D(64, (6, 6), padding='same')(x)
x=SReLU()(x)
x=MaxPooling2D(pool_size=(2,2))(x)

x=Conv2D(128, (3, 3), padding='same')(x)
x=SReLU()(x)
x=MaxPooling2D(pool_size=(2,2))(x)
x=Dropout(0.2)(x)
x=Flatten()(x)
x=Dense(num_classes)(x)
output=Activation('softmax')(x)
model=Model([inputs], output)

In [57]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 100, 100, 1)       0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 100, 100, 16)      1040      
_________________________________________________________________
s_re_lu_25 (SReLU)           (None, 100, 100, 16)      640000    
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 50, 50, 32)        25120     
_________________________________________________________________
s_re_lu_26 (SReLU)           (None, 50, 50, 32)        320000    
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 25, 25, 32)        0         
__________

In [58]:
opt=keras.optimizers.RMSprop(lr=1e-3)

In [59]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [60]:
import h5py
from keras.callbacks import ModelCheckpoint
filepath='E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/weights-improvement-{epoch:2d}-{val_acc:.2f}.hdf5'
checkpoint=ModelCheckpoint(filepath, monitor='val_acc', verbose=1, mode='max')

In [61]:
# hist=model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
#                         epochs=epochs,
#                         validation_data=(x_test, y_test),
#                         callbacks=[checkpoint]
#                )

In [62]:
hist=model.fit(x_train, y_train, 
               batch_size=batch_size,
               epochs=epochs,
               validation_data=(x_test, y_test),
               callbacks=[checkpoint]
               )

Train on 190 samples, validate on 95 samples
Epoch 1/25
190/190 [==============================] - 17s 89ms/step - loss: 3.1667 - acc: 0.1263 - val_loss: 2.4009 - val_acc: 0.2842

Epoch 00001: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/weights-improvement- 1-0.28.hdf5
Epoch 2/25
190/190 [==============================] - 15s 77ms/step - loss: 1.6757 - acc: 0.4211 - val_loss: 1.2220 - val_acc: 0.5474

Epoch 00002: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/weights-improvement- 2-0.55.hdf5
Epoch 3/25
190/190 [==============================] - 14s 74ms/step - loss: 0.9697 - acc: 0.6684 - val_loss: 0.8039 - val_acc: 0.6632

Epoch 00003: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/weights-improvement- 3-0.66.hdf5
Epoch 4/25
190/190 [==============================] - 13s 71ms/step - loss: 0.5988 - acc: 0.7842 - val_loss: 0.7483 - val_acc: 0.7053

Epoch 00004: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/E

In [63]:
score=model.evaluate(x_test, y_test)

95/95 [==============================] - 2s 18ms/step


In [64]:
score[1]

0.92631579010110154

In [65]:
y_pred=model.predict(x_test)# for dataset of 12 characters
print (y_pred)
y_pred=numpy.argmax(y_pred, axis=1)
print (y_pred)

[[  3.47644504e-12   3.22111370e-13   1.29508214e-06 ...,   1.06734087e-05
    3.26745706e-08   8.32946068e-09]
 [  7.28400795e-16   3.30637101e-10   1.68200426e-10 ...,   1.14621976e-07
    9.99998331e-01   4.98533259e-09]
 [  3.87072881e-15   2.40499415e-16   9.99984503e-01 ...,   6.41880771e-10
    6.85059440e-11   3.08459508e-12]
 ..., 
 [  3.08492749e-18   5.99201799e-11   2.97410288e-16 ...,   1.00000000e+00
    3.15580166e-11   6.59199222e-17]
 [  6.66140575e-25   2.98017407e-19   2.73270586e-20 ...,   1.07321780e-19
    2.37718363e-24   1.28491983e-26]
 [  6.10089541e-15   4.28373394e-14   3.60455582e-14 ...,   5.25120049e-06
    1.55909096e-12   1.37469159e-12]]
[26 35  2 35  7 35 31 24 11 18 32 22 14 22 35 36 31 27 30 32  7 18 17  1 28
 27 30 28 17 28 35 17  2 20 17  7 35 17 24 36 17 27  7  2 24 34  2 36 35 28
 27 36  7 11 11 11 28 35 22 35 36 24 18 34 32 31 26 24 18 18  7  1 31 20 34
 26 14 18 31 22 11 36 18 24 27 11 36 18  2  2  7 28 34 30 27]


In [68]:
model.load_weights("E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/weights-improvement-18-0.97.hdf5")# f1

In [69]:
score=model.evaluate(x_test, y_test)

95/95 [==============================] - 2s 18ms/step


In [71]:
score[1]

0.9684210432203193

In [72]:
y_pred=model.predict(x_test)# for dataset of 12 characters
print (y_pred)
y_pred=numpy.argmax(y_pred, axis=1)
print (y_pred)

[[  7.30041808e-12   1.70339177e-11   1.20046952e-05 ...,   7.01342424e-06
    1.30440708e-06   2.15105072e-08]
 [  1.08027041e-14   3.56684033e-08   4.29111857e-09 ...,   9.36162394e-07
    9.99993801e-01   3.34069199e-08]
 [  1.16249024e-13   1.86050489e-13   9.99940038e-01 ...,   2.90433633e-08
    1.45696504e-07   6.57217059e-11]
 ..., 
 [  7.75159720e-18   3.08780046e-09   6.50223758e-14 ...,   9.99999285e-01
    3.90982091e-09   1.29641251e-15]
 [  1.39784416e-21   2.01232582e-15   1.18056456e-15 ...,   1.26308627e-16
    1.54347915e-17   3.31020879e-20]
 [  3.93720511e-13   4.21661074e-11   8.54988649e-11 ...,   2.60360266e-05
    3.41155526e-09   2.55456073e-10]]
[26 35  2 35  7 35 31 24 11 18 32 22 14 22 35 36 31 27 32 32  7 18 17  1 28
 27 30 28 14 28 35 17  2 20 17  7 35 34 24 36 17 26  7  2 24 34  2 36 35 28
 27 36  7 11 11 11 28 35 22 35 36 24  2 34 32 31 26 24 18 18  7  1 31 20 34
 26 32 18 31 22 11 36 18 24 27 11 36 18  2  2  7 28 34 30 27]


In [74]:
print(confusion_matrix(numpy.argmax(y_test, axis=1),y_pred))

[[2 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 3 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 2 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 9 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 7]]


In [75]:

print (classification_report(numpy.argmax(y_test, axis=1), y_pred))

             precision    recall  f1-score   support

          1       1.00      0.67      0.80         3
          2       1.00      1.00      1.00         7
          7       1.00      1.00      1.00         7
         11       1.00      1.00      1.00         6
         14       0.50      1.00      0.67         1
         17       0.75      0.75      0.75         4
         18       0.86      1.00      0.92         6
         20       1.00      0.67      0.80         3
         22       1.00      1.00      1.00         4
         24       1.00      1.00      1.00         6
         26       1.00      1.00      1.00         4
         27       1.00      1.00      1.00         5
         28       1.00      1.00      1.00         6
         30       1.00      1.00      1.00         2
         31       1.00      1.00      1.00         5
         32       1.00      1.00      1.00         5
         34       1.00      1.00      1.00         5
         35       1.00      1.00      1.00   

In [76]:
import pandas

pandas.DataFrame(hist.history).to_csv("E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/Figure/History-97m.csv")        
# visualizing losses and accuracy
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']
xc=range(epochs)

In [77]:
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages

pp=PdfPages("E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/Figure/Loss-97m.pdf")
plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
print (plt.style.available)
plt.style.use(['classic'])
plt.savefig(pp, format='pdf')
pp.close()

['bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'Solarize_Light2', '_classic_test']


In [78]:
pp=PdfPages("E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/Figure/Accuracy-97m.pdf")
plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['bmh'])
plt.savefig(pp, format='pdf')
pp.close()